In [1]:
%%capture --no-stderr
%pip install --upgrade --quiet langchain langchain-community langchainhub langchain-chroma beautifulsoup4
!pip install -q langchain_openai pdfplumber

In [2]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "RAG_With_Memory"

In [3]:
from google.colab import auth
import os

OPENAI_API_KEY = "My api key"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

In [6]:
# Initialize OpenAI embeddings and model
openai_embeddings = OpenAIEmbeddings()
model = ChatOpenAI(model="gpt-4o-mini", convert_system_message_to_human=True)

In [8]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

# Initialize OpenAI model and embeddings
openai_embeddings = OpenAIEmbeddings()
model = ChatOpenAI(model="gpt-4o-mini")  # Use the correct model name for OpenAI

# Test the model
response = model("hi")
print(response.content) # Extract the response content


Hello! How can I assist you today?


In [9]:
import os
import pdfplumber
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.schema import Document
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

In [19]:
# PDF loader setup
def load_documents(pdf_path=None):
    # Load PDF documents if path is provided
    pdf_docs = []
    if pdf_path:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    pdf_docs.append(Document(page_content=text, metadata={"source": pdf_path}))

    # return web_docs + pdf_docs
    return pdf_docs

In [20]:

doc = load_documents(pdf_path="/content/Multiple crops in one.pdf")  # Update path to your PDF

In [21]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(doc)

In [22]:
vectorstore = Chroma.from_documents(documents=splits, embedding=openai_embeddings)
retriever = vectorstore.as_retriever()


In [23]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, say that you don't know. "
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

In [24]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [25]:
question_answering_chain = create_stuff_documents_chain(model, chat_prompt)

In [26]:
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

In [27]:
def get_answer(query_text, retriever, question_answering_chain):
    # Retrieve relevant documents
    results = retriever.get_relevant_documents(query_text)

    if len(results) == 0:
        # No relevant results in document by user
        return "Unable to find the answer in the document you provided.", []

    # Extract relevant context from documents
    context_text = "\n\n---\n\n".join([doc.page_content for doc in results])
    prompt_template = ChatPromptTemplate.from_template(system_prompt)
    prompt = prompt_template.format(context=context_text, question=query_text)

    # Use OpenAI for the model response
    response = model.predict(prompt)

    # Debugging: Print the response to see its structure
    print("Model Response:", response)

    # If the response is a string, we can return it directly
    if isinstance(response, str):
        response_text = response
    else:
        # Otherwise, extract content assuming it's a dictionary-like object
        response_text = response["choices"][0]["message"]["content"]

    # Fetch metadata to show sources of the answer
    sources = [doc.metadata.get("source", None) for doc in results]
    return response_text, sources


In [28]:
# Example usage
response_text, sources = get_answer("what are managing tips for Soybean?", retriever, question_answering_chain)
print("Answer:", response_text)
print("Sources:", sources)

<ipython-input-27-71d95a124953>:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query_text)
<ipython-input-27-71d95a124953>:15: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = model.predict(prompt)


Model Response: For effective control of soybean cyst nematode, avoid planting the same variety in the same field twice and rotate with varieties that offer different sources of resistance. This practice is rated as a highly effective control measure. Additionally, managing soil and seed quality is crucial for overall disease resistance in soybean crops.
Answer: For effective control of soybean cyst nematode, avoid planting the same variety in the same field twice and rotate with varieties that offer different sources of resistance. This practice is rated as a highly effective control measure. Additionally, managing soil and seed quality is crucial for overall disease resistance in soybean crops.
Sources: ['/content/Multiple crops in one.pdf', '/content/Multiple crops in one.pdf', '/content/Multiple crops in one.pdf', '/content/Multiple crops in one.pdf']


In [29]:
# Example usage
response_text, sources = get_answer("what are corn managing tips?", retriever, question_answering_chain)
print("Answer:", response_text)
print("Sources:", sources)

Model Response: To manage corn diseases effectively, an integrated approach is necessary, including the use of disease-resistant hybrids, crop rotations, proper tillage, balanced fertility, and pest control. Weekly inspections for signs of storage molds and maintaining appropriate moisture levels are also crucial for stored corn. Applying fungicides and other cultural practices can help limit disease and yield losses in corn production.
Answer: To manage corn diseases effectively, an integrated approach is necessary, including the use of disease-resistant hybrids, crop rotations, proper tillage, balanced fertility, and pest control. Weekly inspections for signs of storage molds and maintaining appropriate moisture levels are also crucial for stored corn. Applying fungicides and other cultural practices can help limit disease and yield losses in corn production.
Sources: ['/content/Multiple crops in one.pdf', '/content/Multiple crops in one.pdf', '/content/Multiple crops in one.pdf', '/

In [30]:
# Example usage
response_text, sources = get_answer("how to manage fertility in Wheat crop management?", retriever, question_answering_chain)
print("Answer:", response_text)
print("Sources:", sources)

Model Response: To manage wheat diseases effectively, it is crucial to integrate resistant varieties, high-quality seed, and appropriate fungicide treatments. Soil health and fertility also play a significant role in reducing disease severity, especially for diseases like Fusarium head blight and powdery mildew. Additionally, timely planting and crop rotation are essential for controlling diseases and preventing their recurrence.
Answer: To manage wheat diseases effectively, it is crucial to integrate resistant varieties, high-quality seed, and appropriate fungicide treatments. Soil health and fertility also play a significant role in reducing disease severity, especially for diseases like Fusarium head blight and powdery mildew. Additionally, timely planting and crop rotation are essential for controlling diseases and preventing their recurrence.
Sources: ['/content/Multiple crops in one.pdf', '/content/Multiple crops in one.pdf', '/content/Multiple crops in one.pdf', '/content/Multip

In [37]:
# Example usage
response_text, sources = get_answer("what stewart's bacterial disease in corn?", retriever, question_answering_chain)
print("Answer:", response_text)
print("Sources:", sources)

Model Response: To manage corn diseases effectively in Illinois, an integrated approach is essential, incorporating disease-resistant hybrids, crop rotations, and balanced fertility. Specific practices can vary in effectiveness, with measures like using high-quality seeds and controlling pests being crucial. For optimal outcomes, refer to Table 14.3 for a detailed list of diseases and control measures.
Answer: To manage corn diseases effectively in Illinois, an integrated approach is essential, incorporating disease-resistant hybrids, crop rotations, and balanced fertility. Specific practices can vary in effectiveness, with measures like using high-quality seeds and controlling pests being crucial. For optimal outcomes, refer to Table 14.3 for a detailed list of diseases and control measures.
Sources: ['/content/Multiple crops in one.pdf', '/content/Multiple crops in one.pdf', '/content/Multiple crops in one.pdf', '/content/Multiple crops in one.pdf']


In [31]:
response = rag_chain.invoke({"input": "what are fungicides?"})
print(response["answer"])

Fungicides are chemicals used to prevent or control fungal diseases in plants. They can be classified into contact fungicides, which remain on the plant's surface, and systemic fungicides, which are absorbed into the plant tissue and can move within it. Some fungicides have preventative activity, while others may still be effective after the fungal pathogen has invaded the plant.


In [38]:
from langchain.chains import create_history_aware_retriever

retriever_prompt = (
    "Given a chat history and the latest user question which might reference context in the chat history, "
    "formulate a standalone question which can be understood without the chat history. "
    "Do NOT answer the question, just reformulate it if needed and otherwise return it as is."
)

In [39]:
contextualize_q_prompt  = ChatPromptTemplate.from_messages(
    [
        ("system", retriever_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

In [40]:
history_aware_retriever = create_history_aware_retriever(model, retriever, contextualize_q_prompt)

In [41]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [42]:
question_answer_chain = create_stuff_documents_chain(model, qa_prompt)


In [43]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [44]:
chat_history = []

In [45]:
question1 = "what is Alfalfa crop?"

In [46]:
message1 = rag_chain.invoke({"input": question1, "chat_history": chat_history})
print(message1["answer"])


Alfalfa is a perennial flowering plant in the legume family, commonly grown as a forage crop for livestock. It is valued for its high protein content and nutritional quality, making it an excellent feed for dairy and beef cattle, sheep, and horses. Alfalfa is also known for its ability to improve soil health through nitrogen fixation.


In [47]:
chat_history.extend(
    [
        HumanMessage(content=question1),
        AIMessage(content=message1["answer"]),
    ]
)


In [48]:
chat_history

[HumanMessage(content='what is Alfalfa crop?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Alfalfa is a perennial flowering plant in the legume family, commonly grown as a forage crop for livestock. It is valued for its high protein content and nutritional quality, making it an excellent feed for dairy and beef cattle, sheep, and horses. Alfalfa is also known for its ability to improve soil health through nitrogen fixation.', additional_kwargs={}, response_metadata={})]

In [49]:
second_question = "What are common ways of doing it?"
message2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})


In [50]:
print(message2["answer"])


Common ways to manage alfalfa crops include using crop rotation with nonlegumes, selecting disease-resistant varieties, applying proper fertilization, and controlling insects and weeds. Timely cutting and avoiding overgrazing during critical growth periods are also important practices. Additionally, integrating the use of fungicides and maintaining soil health through proper pH and drainage are effective management strategies.


In [51]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [52]:
store = {}

In [53]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [54]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)


In [56]:
conversational_rag_chain.invoke(
    {"input": "What is fungicide resistance management?"},
    config={
        "configurable": {"session_id": "abc123"}
    },
)["answer"]

'Fungicide resistance management involves strategies and practices aimed at preventing or reducing the development of resistance in fungal pathogens to fungicides. This includes applying fungicides only when necessary, using fungicides with different modes of action, following label recommendations, and integrating cultural practices such as growing resistant varieties and using disease-free seeds. The goal is to prolong the effectiveness of fungicides and minimize the selection pressure on pathogen populations.'

In [57]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?', additional_kwargs={}, response_metadata={}), AIMessage(content='Task decomposition is the process of breaking down a complex task into smaller, more manageable sub-tasks or components. This approach helps to simplify the execution and understanding of the overall task, making it easier to tackle each part systematically. It is often used in project management, programming, and problem-solving to enhance efficiency and clarity.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is fungicide resistance management?', additional_kwargs={}, response_metadata={}), AIMessage(content='Fungicide resistance management involves strategies and practices aimed at preventing or reducing the development of resistance in fungal pathogens to fungicides. This includes applying fungicides only when necessary, using fungicides with different modes of action, following label recommenda

In [58]:
conversational_rag_chain.invoke(
    {"input": "What are steps in fungicide resistance management?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]


'The steps in fungicide resistance management include: \n\n1. Apply fungicides only when necessary, based on field scouting and disease risk factors.\n2. Use fungicides with different modes of action and rotate them if multiple applications are needed in a season.\n3. Grow resistant plant varieties, plant high-quality disease-free seeds, and provide a well-drained seedbed.\n4. Follow label recommendations regarding application rates and restrictions.'

In [59]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What is Task Decomposition?

AI: Task decomposition is the process of breaking down a complex task into smaller, more manageable sub-tasks or components. This approach helps to simplify the execution and understanding of the overall task, making it easier to tackle each part systematically. It is often used in project management, programming, and problem-solving to enhance efficiency and clarity.

User: What is fungicide resistance management?

AI: Fungicide resistance management involves strategies and practices aimed at preventing or reducing the development of resistance in fungal pathogens to fungicides. This includes applying fungicides only when necessary, using fungicides with different modes of action, following label recommendations, and integrating cultural practices such as growing resistant varieties and using disease-free seeds. The goal is to prolong the effectiveness of fungicides and minimize the selection pressure on pathogen populations.

User: What are steps in

In [61]:
conversational_rag_chain.invoke(
    {"input": "What is principle of plant disease control?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'The principles of plant disease control involve strategies designed to prevent or minimize the impact of diseases on crops. Key principles include accurate diagnosis of the disease, using resistant plant varieties, implementing cultural practices (such as crop rotation and proper sanitation), and applying chemical controls like fungicides when necessary. Integrated Pest Management (IPM) is often emphasized, combining multiple approaches to achieve effective disease management while reducing negative environmental impacts.'

In [62]:
store


{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?', additional_kwargs={}, response_metadata={}), AIMessage(content='Task decomposition is the process of breaking down a complex task into smaller, more manageable sub-tasks or components. This approach helps to simplify the execution and understanding of the overall task, making it easier to tackle each part systematically. It is often used in project management, programming, and problem-solving to enhance efficiency and clarity.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is fungicide resistance management?', additional_kwargs={}, response_metadata={}), AIMessage(content='Fungicide resistance management involves strategies and practices aimed at preventing or reducing the development of resistance in fungal pathogens to fungicides. This includes applying fungicides only when necessary, using fungicides with different modes of action, following label recommenda

In [63]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What is Task Decomposition?

AI: Task decomposition is the process of breaking down a complex task into smaller, more manageable sub-tasks or components. This approach helps to simplify the execution and understanding of the overall task, making it easier to tackle each part systematically. It is often used in project management, programming, and problem-solving to enhance efficiency and clarity.

User: What is fungicide resistance management?

AI: Fungicide resistance management involves strategies and practices aimed at preventing or reducing the development of resistance in fungal pathogens to fungicides. This includes applying fungicides only when necessary, using fungicides with different modes of action, following label recommendations, and integrating cultural practices such as growing resistant varieties and using disease-free seeds. The goal is to prolong the effectiveness of fungicides and minimize the selection pressure on pathogen populations.

User: What are steps in

My more queries to extend the explanation.

In [64]:
conversational_rag_chain.invoke(
    {"input": "What are Alfalfa diseases that reduce yields in Illinois and the relative effectiveness of various control measures?"},
    config={
        "configurable": {"session_id": "abc123"}
    },
)["answer"]

'Alfalfa diseases that reduce yields in Illinois include bacterial wilt, anthracnose, Phytophthora root rot, Fusarium wilt, and various fungal crown and root rots. Control measures vary in effectiveness, with the use of resistant varieties being the most economical and efficient method. Crop rotation, proper fertilization, timely cutting, and controlling insects and weeds are also recommended agronomic practices to manage these diseases.'

In [65]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?', additional_kwargs={}, response_metadata={}), AIMessage(content='Task decomposition is the process of breaking down a complex task into smaller, more manageable sub-tasks or components. This approach helps to simplify the execution and understanding of the overall task, making it easier to tackle each part systematically. It is often used in project management, programming, and problem-solving to enhance efficiency and clarity.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is fungicide resistance management?', additional_kwargs={}, response_metadata={}), AIMessage(content='Fungicide resistance management involves strategies and practices aimed at preventing or reducing the development of resistance in fungal pathogens to fungicides. This includes applying fungicides only when necessary, using fungicides with different modes of action, following label recommenda